In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
import numpy as np
from PIL import Image

# Custom ImageDataGenerator class
class CustomImageDataGenerator(ImageDataGenerator):
    def flow_from_directory(self, directory, *args, **kwargs):
        generator = super().flow_from_directory(directory, *args, **kwargs)
        self.target_size = kwargs.get('target_size', (224, 224))
        self.num_classes = generator.num_classes
        self.filepaths = generator.filepaths
        self.labels = generator.classes
        return generator

# Data Generators
train_datagen = CustomImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)

valid_datagen = CustomImageDataGenerator()
test_datagen = CustomImageDataGenerator()

# Directories for subset data
train_subset_dir = "/Users/Barbara/Downloads/food11/training"
val_subset_dir = "/Users/Barbara/Downloads/food11/validation"
test_subset_dir = "/Users/Barbara/Downloads/food11/evaluation"

# Generators
train_generator = train_datagen.flow_from_directory(train_subset_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(val_subset_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_subset_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

# Get the number of classes from the train generator
num_classes = train_generator.num_classes

# Load the VGG16 model without the top layers and add custom layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the VGG16 base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Convert generators to tf.data.Dataset
def generator_to_tfdata(generator):
    def gen():
        for x_batch, y_batch in generator:
            yield x_batch, y_batch

    return tf.data.Dataset.from_generator(gen, (tf.float32, tf.float32))

# Convert generators to tf.data.Dataset with prefetching and better optimization
train_dataset = generator_to_tfdata(train_generator).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
valid_dataset = generator_to_tfdata(valid_generator).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = generator_to_tfdata(test_generator).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Custom training loop
num_epochs = 30
batch_size = 32

@tf.function
def train_step(model, x_batch, y_batch):
    with tf.GradientTape() as tape:
        predictions = model(x_batch, training=True)
        loss = tf.keras.losses.categorical_crossentropy(y_batch, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss, predictions

@tf.function
def valid_step(model, x_batch, y_batch):
    predictions = model(x_batch, training=False)
    loss = tf.keras.losses.categorical_crossentropy(y_batch, predictions)
    return loss, predictions

for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")
    
    # Training
    train_loss = 0
    train_accuracy = 0
    train_batches = 0
    
    for x_batch, y_batch in train_dataset:
        loss, predictions = train_step(model, x_batch, y_batch)
        train_loss += tf.reduce_mean(loss)
        train_accuracy += tf.reduce_mean(tf.keras.metrics.categorical_accuracy(y_batch, predictions))
        train_batches += 1
        
        if train_batches % 10 == 0:
            print(f"Batch {train_batches}: Loss = {train_loss/train_batches:.4f}, Accuracy = {train_accuracy/train_batches:.4f}")
        if train_batches >= len(train_generator):  # Ensure loop breaks after all batches are processed
            break
    
    train_loss /= train_batches
    train_accuracy /= train_batches
    print(f"Training - Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}")
    
    # Validation
    val_loss = 0
    val_accuracy = 0
    val_batches = 0
    
    for x_batch, y_batch in valid_dataset:
        loss, predictions = valid_step(model, x_batch, y_batch)
        val_loss += tf.reduce_mean(loss)
        val_accuracy += tf.reduce_mean(tf.keras.metrics.categorical_accuracy(y_batch, predictions))
        val_batches += 1
    
        if val_batches >= len(valid_generator):  # Ensure loop breaks after all batches are processed
            break
    
    val_loss /= val_batches
    val_accuracy /= val_batches
    print(f"Validation - Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}")

# Save the model
model.save('/Users/Barbara/Desktop/Ironhack/Final_Project/food_recognition_model4.h5')

# Evaluate the model on the test set
test_loss = 0
test_accuracy = 0
test_batches = 0
all_predictions = []
all_true_labels = []

for x_batch, y_batch in test_dataset:
    loss, predictions = valid_step(model, x_batch, y_batch)
    test_loss += tf.reduce_mean(loss)
    test_accuracy += tf.reduce_mean(tf.keras.metrics.categorical_accuracy(y_batch, predictions))
    
    all_predictions.extend(np.argmax(predictions, axis=1))
    all_true_labels.extend(np.argmax(y_batch, axis=1))
    
    test_batches += 1

    if test_batches >= len(test_generator):  # Ensure loop breaks after all batches are processed
        break

test_loss /= test_batches
test_accuracy /= test_batches
print(f"Test - Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}")

Found 9866 images belonging to 11 classes.
Found 3430 images belonging to 11 classes.
Found 3347 images belonging to 11 classes.


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_6      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 11)             │         1,419 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,781,771 (56.39 MB)

 Trainable params: 67,083 (262.04 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

Epoch 1/30
Batch 10: Loss = 10.0156, Accuracy = 0.1125
Batch 20: Loss = 7.2928, Accuracy = 0.1609
Batch 30: Loss = 6.0095, Accuracy = 0.1594
Batch 40: Loss = 5.1409, Accuracy = 0.1750
Batch 50: Loss = 4.5941, Accuracy = 0.1813
Batch 60: Loss = 4.2105, Accuracy = 0.1958
Batch 70: Loss = 3.9240, Accuracy = 0.2076
Batch 80: Loss = 3.7129, Accuracy = 0.2141
Batch 90: Loss = 3.5457, Accuracy = 0.2215
Batch 100: Loss = 3.4039, Accuracy = 0.2266
Batch 110: Loss = 3.2919, Accuracy = 0.2315
Batch 120: Loss = 3.1882, Accuracy = 0.2393
Batch 130: Loss = 3.1101, Accuracy = 0.2438
Batch 140: Loss = 3.0470, Accuracy = 0.2433
Batch 150: Loss = 2.9817, Accuracy = 0.2473
Batch 160: Loss = 2.9268, Accuracy = 0.2514
Batch 170: Loss = 2.8788, Accuracy = 0.2540
Batch 180: Loss = 2.8418, Accuracy = 0.2564
Batch 190: Loss = 2.7989, Accuracy = 0.2581
Batch 200: Loss = 2.7618, Accuracy = 0.2606
Batch 210: Loss = 2.7195, Accuracy = 0.2665
Batch 220: Loss = 2.6822, Accuracy = 0.2702
Batch 230: Loss = 2.6557, Acc

Validation - Loss: 0.7694, Accuracy: 0.7632
Test - Loss: 0.6872, Accuracy: 0.7740


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np

# Custom ImageDataGenerator class
class CustomImageDataGenerator(ImageDataGenerator):
    def flow_from_directory(self, directory, *args, **kwargs):
        generator = super().flow_from_directory(directory, *args, **kwargs)
        self.target_size = kwargs.get('target_size', (224, 224))
        self.num_classes = generator.num_classes
        self.filepaths = generator.filepaths
        self.labels = generator.classes
        return generator

# Directories for subset data
test_subset_dir = "/Users/Barbara/Downloads/food11/evaluation"

# Data Generators
test_datagen = CustomImageDataGenerator()
test_generator = test_datagen.flow_from_directory(test_subset_dir, target_size=(224, 224), batch_size=32, class_mode='categorical', shuffle=False)

# Load the trained model
model = tf.keras.models.load_model('/Users/Barbara/Desktop/Ironhack/Final_Project/food_recognition_model4.h5')

# Use model.evaluate to get the loss and accuracy
loss, accuracy = model.evaluate(test_generator)
print(f"Overall Test Loss: {loss:.4f}")
print(f"Overall Test Accuracy: {accuracy:.4f}")

# Get predictions and true labels in one pass
test_steps = test_generator.samples // test_generator.batch_size
predictions = model.predict(test_generator, steps=test_steps)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes[:test_steps * test_generator.batch_size]

# Calculate and print classification report
target_names = list(test_generator.class_indices.keys())
report = classification_report(true_classes, predicted_classes, target_names=target_names, output_dict=True)

# Calculate accuracy for each class
accuracies = {}
for label, metrics in report.items():
    if label not in target_names:
        continue
    accuracies[label] = metrics['precision']  # Using precision as class accuracy

# Print accuracy for each class
print("Class-wise Accuracy:")
for class_name, accuracy in accuracies.items():
    print(f"Accuracy for class {class_name}: {accuracy:.2f}")

# Optional: Print the full classification report for additional details
print("\nFull Classification Report:")
print(classification_report(true_classes, predicted_classes, target_names=target_names))

Found 3347 images belonging to 11 classes.


2024-07-11 12:44:33.172102: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2024-07-11 12:44:33.172128: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2024-07-11 12:44:33.172134: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2024-07-11 12:44:33.172148: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-11 12:44:33.172158: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-07-11 12:44:33.533416: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/Applications/Utilities/ana

105/105 ━━━━━━━━━━━━━━━━━━━━ 21s 198ms/step - accuracy: 0.6757 - loss: 0.9397
Overall Test Loss: 0.6860
Overall Test Accuracy: 0.7747
104/104 ━━━━━━━━━━━━━━━━━━━━ 20s 189ms/step
Class-wise Accuracy:
Accuracy for class Bread: 0.70
Accuracy for class Dairy product: 0.80
Accuracy for class Dessert: 0.72
Accuracy for class Egg: 0.68
Accuracy for class Fried food: 0.84
Accuracy for class Meat: 0.66
Accuracy for class Noodles-Pasta: 0.96
Accuracy for class Rice: 0.90
Accuracy for class Seafood: 0.76
Accuracy for class Soup: 0.93
Accuracy for class Vegetable-Fruit: 0.85

Full Classification Report:
                 precision    recall  f1-score   support

          Bread       0.70      0.62      0.66       368
  Dairy product       0.80      0.45      0.57       148
        Dessert       0.72      0.71      0.71       500
            Egg       0.68      0.67      0.68       335
     Fried food       0.84      0.60      0.70       287
           Meat       0.66      0.90      0.76       432
 

In [6]:
data = classification_report(true_classes, predicted_classes, target_names=target_names, output_dict=True)



In [7]:
import pandas as pd
metrics = pd.DataFrame(data)

In [8]:
metrics

,Bread,Dairy product,Dessert,Egg,Fried food,Meat,Noodles-Pasta,Rice,Seafood,Soup,Vegetable-Fruit,accuracy,macro avg,weighted avg
precision,0.704969,0.795181,0.716298,0.676647,0.843137,0.663823,0.958333,0.903226,0.763636,0.927451,0.853333,0.773738,0.800549,0.779535
recall,0.616848,0.445946,0.712000,0.674627,0.599303,0.900463,0.938776,0.875000,0.831683,0.946000,0.905660,0.773738,0.767846,0.773738
f1-score,0.657971,0.571429,0.714142,0.675635,0.700611,0.764244,0.948454,0.888889,0.796209,0.936634,0.878719,0.773738,0.775721,0.769818
support,368.000000,148.000000,500.000000,335.000000,287.000000,432.000000,147.000000,96.000000,303.000000,500.000000,212.000000,0.773738,3328.000000,3328.000000


In [10]:
import pandas as pd

# Create the DataFrame
data = {
    'Food Category': ['Bread', 'Dairy product', 'Dessert', 'Egg', 'Fried food', 'Meat', 'Noodles-Pasta', 'Rice', 'Seafood', 'Soup', 'Vegetable-Fruit', 'accuracy', 'macro avg', 'weighted avg'],
    'precision': [0.704969, 0.795181, 0.716298, 0.676647, 0.843137, 0.663823, 0.958333, 0.903226, 0.763636, 0.927451, 0.853333, 0.773738, 0.800549, 0.779535],
    'recall': [0.616848, 0.445946, 0.712000, 0.674627, 0.599303, 0.900463, 0.938776, 0.875000, 0.831683, 0.946000, 0.905660, 0.773738, 0.767846, 0.773738],
    'f1-score': [0.657971, 0.571429, 0.714142, 0.675635, 0.700611, 0.764244, 0.948454, 0.888889, 0.796209, 0.936634, 0.878719, 0.773738, 0.775721, 0.769818],
    'support': [368, 148, 500, 335, 287, 432, 147, 96, 303, 500, 212, 0.773738, 3328, 3328]
}

df_metrics = pd.DataFrame(data)

In [11]:
df_metrics

,Food Category,precision,recall,f1-score,support
0,Bread,0.704969,0.616848,0.657971,368.000000
1,Dairy product,0.795181,0.445946,0.571429,148.000000
2,Dessert,0.716298,0.712000,0.714142,500.000000
3,Egg,0.676647,0.674627,0.675635,335.000000
4,Fried food,0.843137,0.599303,0.700611,287.000000
5,Meat,0.663823,0.900463,0.764244,432.000000
6,Noodles-Pasta,0.958333,0.938776,0.948454,147.000000
7,Rice,0.903226,0.875000,0.888889,96.000000
8,Seafood,0.763636,0.831683,0.796209,303.000000
9,Soup,0.927451,0.946000,0.936634,500.000000


In [12]:
df_metrics.to_csv('metrics.csv',index=False )

In [17]:
from sklearn.metrics import confusion_matrix

In [22]:
confusion_matrix(y_true=true_classes, y_pred=predicted_classes)

array([[227,   0,  26,  40,  17,  38,   0,   2,  13,   4,   1],
       [  8,  66,  44,   5,   3,   9,   0,   0,   7,   4,   2],
       [ 13,  10, 356,  28,   4,  45,   1,   1,  17,  15,  10],
       [ 39,   1,  14, 226,   3,  27,   1,   3,  14,   2,   5],
       [ 23,   3,  21,   8, 172,  46,   0,   1,  11,   1,   1],
       [  7,   0,  14,   7,   3, 389,   1,   0,   7,   4,   0],
       [  0,   0,   0,   4,   1,   0, 138,   0,   0,   1,   3],
       [  1,   0,   1,   1,   0,   1,   1,  84,   1,   2,   4],
       [  2,   1,   9,  10,   0,  17,   1,   1, 252,   4,   6],
       [  1,   1,  10,   2,   0,   8,   0,   1,   3, 473,   1],
       [  1,   1,   2,   3,   1,   6,   1,   0,   5,   0, 192]])

In [24]:
predicted_classes[:5]

array([3, 0, 0, 0, 0])

In [33]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np

# Custom ImageDataGenerator class
class CustomImageDataGenerator(ImageDataGenerator):
    def flow_from_directory(self, directory, *args, **kwargs):
        generator = super().flow_from_directory(directory, *args, **kwargs)
        self.target_size = kwargs.get('target_size', (224, 224))
        self.num_classes = generator.num_classes
        self.filepaths = generator.filepaths
        self.labels = generator.classes
        return generator

# Directories for subset data
test_subset_dir = "/Users/Barbara/Downloads/food11/bread"

# Data Generators
test_datagen = CustomImageDataGenerator()
test_generator = test_datagen.flow_from_directory(test_subset_dir, target_size=(224, 224), batch_size=32, class_mode='categorical', shuffle=False)

# Load the trained model
model = tf.keras.models.load_model('/Users/Barbara/Desktop/Ironhack/Final_Project/food_recognition_model4.h5')

# Use model.evaluate to get the loss and accuracy


# Get predictions and true labels in one pass
test_steps = test_generator.samples // test_generator.batch_size
predictions = model.predict(test_generator, steps=test_steps)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes[:test_steps * test_generator.batch_size]



# Calculate accuracy for each class
accuracies = {}
for label, metrics in report.items():
    if label not in target_names:
        continue
    accuracies[label] = metrics['precision']  # Using precision as class accuracy

# Print accuracy for each class
print("Class-wise Accuracy:")
for class_name, accuracy in accuracies.items():
    print(f"Accuracy for class {class_name}: {accuracy:.2f}")

predicted_classes



Found 1 images belonging to 3 classes.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
Class-wise Accuracy:


array([0])